In [ ]:
Mod 4 HW

In [4]:
import pandas as pd
from pathlib import Path

In [5]:
school_data = Path("../Resources/schools_complete.csv")
student_data = Path("../Resources/students_complete.csv")

In [8]:
school_data_df = pd.read_csv(school_data)
school_data_df.head()

,School ID,school_name,type,size,budget
0,0,Huang High School,District,2917,1910635
1,1,Figueroa High School,District,2949,1884411
2,2,Shelton High School,Charter,1761,1056600
3,3,Hernandez High School,District,4635,3022020
4,4,Griffin High School,Charter,1468,917500


In [10]:
student_data_df = pd.read_csv(student_data)
student_data_df.head()

,Student ID,student_name,gender,grade,school_name,reading_score,math_score
0,0,Paul Bradley,M,9th,Huang High School,66,79
1,1,Victor Smith,M,12th,Huang High School,94,61
2,2,Kevin Rodriguez,M,12th,Huang High School,90,60
3,3,Dr. Richard Scott,M,12th,Huang High School,67,58
4,4,Bonnie Ray,F,9th,Huang High School,97,84


In [42]:

# Define spending bins and labels
spending_bins = [0, 585, 630, 645, 680]
labels = ["<$585", "$585-630", "$630-645", "$645-680"]

# Calculate the new column "Average Spending Per Student"
school_summary['Average Spending Per Student'] = school_summary['Total School Budget'] / school_summary['Total Students']

# Create a new column to categorize spending based on the bins
school_summary['Spending Ranges (Per Student)'] = pd.cut(school_summary['Average Spending Per Student'], bins=spending_bins, labels=labels)

# Calculate mean scores per spending range
spending_math_scores = school_summary.groupby(['Spending Ranges (Per Student)'])['Average Math Score'].mean()
spending_reading_scores = school_summary.groupby(['Spending Ranges (Per Student)'])['Average Reading Score'].mean()
spending_passing_math = school_summary.groupby(['Spending Ranges (Per Student)'])['% Passing Math'].mean()
spending_passing_reading = school_summary.groupby(['Spending Ranges (Per Student)'])['% Passing Reading'].mean()
overall_passing_spending = school_summary.groupby(['Spending Ranges (Per Student)'])['% Overall Passing'].mean()

# Create the spending_summary DataFrame
spending_summary = pd.DataFrame({
    'Average Math Score': spending_math_scores,
    'Average Reading Score': spending_reading_scores,
    '% Passing Math': spending_passing_math,
    '% Passing Reading': spending_passing_reading,
    '% Overall Passing': overall_passing_spending
})

# Display the spending_summary DataFrame
print(spending_summary)




                               Average Math Score  Average Reading Score  \
Spending Ranges (Per Student)                                              
<$585                                   83.455399              83.933814   
$585-630                                81.899826              83.155286   
$630-645                                78.518855              81.624473   
$645-680                                76.997210              81.027843   

                               % Passing Math  % Passing Reading  \
Spending Ranges (Per Student)                                      
<$585                                     NaN                NaN   
$585-630                                  NaN                NaN   
$630-645                                  NaN                NaN   
$645-680                                  NaN                NaN   

                               % Overall Passing  
Spending Ranges (Per Student)                     
<$585                           

In [40]:

import pandas as pd

# Group student_data_df by school_name and calculate the necessary metrics
school_metrics = student_data_df.groupby('school_name').agg({
    'math_score': 'mean',
    'reading_score': 'mean',
    'Student ID': 'count',
})

# Merge school_metrics with school_data_df on school_name
school_summary = pd.merge(school_data_df, school_metrics, on='school_name')

# Calculate additional metrics
school_summary['Per Student Budget'] = school_summary['budget'] / school_summary['Student ID']

# Calculate % Passing Math, % Passing Reading, and % Overall Passing
passing_math = (student_data_df[student_data_df['math_score'] >= 70]
               .groupby('school_name')['Student ID'].count() / school_metrics['Student ID']) * 100

passing_reading = (student_data_df[student_data_df['reading_score'] >= 70]
                  .groupby('school_name')['Student ID'].count() / school_metrics['Student ID']) * 100

passing_overall = (student_data_df[(student_data_df['math_score'] >= 70) & (student_data_df['reading_score'] >= 70)]
                  .groupby('school_name')['Student ID'].count() / school_metrics['Student ID']) * 100

school_summary['% Passing Math'] = passing_math
school_summary['% Passing Reading'] = passing_reading
school_summary['% Overall Passing'] = passing_overall

# Rename columns for clarity
school_summary = school_summary.rename(columns={
    'school_name': 'School Name',
    'type': 'School Type',
    'Student ID': 'Total Students',
    'budget': 'Total School Budget',
    'math_score': 'Average Math Score',
    'reading_score': 'Average Reading Score'
})

# Create the final DataFrame with the desired columns
school_summary = school_summary[['School Name', 'School Type', 'Total Students', 'Total School Budget', 'Per Student Budget', 'Average Math Score', 'Average Reading Score', '% Passing Math', '% Passing Reading', '% Overall Passing']]

# Display the school summary DataFrame
print(school_summary)



              School Name School Type  Total Students  Total School Budget  \
0       Huang High School    District            2917              1910635   
1    Figueroa High School    District            2949              1884411   
2     Shelton High School     Charter            1761              1056600   
3   Hernandez High School    District            4635              3022020   
4     Griffin High School     Charter            1468               917500   
5      Wilson High School     Charter            2283              1319574   
6     Cabrera High School     Charter            1858              1081356   
7      Bailey High School    District            4976              3124928   
8      Holden High School     Charter             427               248087   
9        Pena High School     Charter             962               585858   
10     Wright High School     Charter            1800              1049400   
11  Rodriguez High School    District            3999           

In [43]:

# Group student_data_df by school_name and calculate the necessary metrics
school_metrics = student_data_df.groupby('school_name').agg({
    'math_score': 'mean',
    'reading_score': 'mean',
    'Student ID': 'count',
})

# Create an empty list to store the results for each school
school_results = []

# Iterate through each school
for school_name, school_data in student_data_df.groupby('school_name'):
    total_students = len(school_data)
    passing_math_percentage = (len(school_data[school_data['math_score'] >= 70]) / total_students) * 100
    passing_reading_percentage = (len(school_data[school_data['reading_score'] >= 70]) / total_students) * 100
    overall_passing_percentage = (len(school_data[(school_data['math_score'] >= 70) & (school_data['reading_score'] >= 70)]) / total_students) * 100

    # Store the results in a dictionary
    school_result = {
        'School Name': school_name,
        '% Passing Math': passing_math_percentage,
        '% Passing Reading': passing_reading_percentage,
        '% Overall Passing': overall_passing_percentage
    }

    # Append the results to the list
    school_results.append(school_result)

# Create a DataFrame from the list of results
school_summary = pd.DataFrame(school_results)

# Display the school summary DataFrame
print(school_summary)




              School Name  % Passing Math  % Passing Reading  \
0      Bailey High School       66.680064          81.933280   
1     Cabrera High School       94.133477          97.039828   
2    Figueroa High School       65.988471          80.739234   
3        Ford High School       68.309602          79.299014   
4     Griffin High School       93.392371          97.138965   
5   Hernandez High School       66.752967          80.862999   
6      Holden High School       92.505855          96.252927   
7       Huang High School       65.683922          81.316421   
8     Johnson High School       66.057551          81.222432   
9        Pena High School       94.594595          95.945946   
10  Rodriguez High School       66.366592          80.220055   
11    Shelton High School       93.867121          95.854628   
12     Thomas High School       93.272171          97.308869   
13     Wilson High School       93.867718          96.539641   
14     Wright High School       93.33333

In [44]:
# Sort the schools by % Overall Passing in descending order and display the top 5 rows
top_schools = school_summary.sort_values(by='% Overall Passing', ascending=False).head(5)

# Display the top 5 schools
print(top_schools)


            School Name  % Passing Math  % Passing Reading  % Overall Passing
1   Cabrera High School       94.133477          97.039828          91.334769
12   Thomas High School       93.272171          97.308869          90.948012
4   Griffin High School       93.392371          97.138965          90.599455
13   Wilson High School       93.867718          96.539641          90.582567
9      Pena High School       94.594595          95.945946          90.540541


In [45]:
# Sort the schools by % Overall Passing in ascending order and display the top 5 rows
bottom_schools = school_summary.sort_values(by='% Overall Passing',ascending=True).head(5)

# Display the bottom 5 schools
print(bottom_schools)


              School Name  % Passing Math  % Passing Reading  \
10  Rodriguez High School       66.366592          80.220055   
2    Figueroa High School       65.988471          80.739234   
7       Huang High School       65.683922          81.316421   
5   Hernandez High School       66.752967          80.862999   
8     Johnson High School       66.057551          81.222432   

    % Overall Passing  
10          52.988247  
2           53.204476  
7           53.513884  
5           53.527508  
8           53.539172  


In [46]:

# Define the binning criteria and group names
bins = [0, 59.9, 69.9, 79.9, 89.9, 100]
group_names = ["F", "D", "C", "B", "A"]

# Create a new column in the student_data_df DataFrame to categorize math scores into bins
student_data_df['Math Score Bins'] = pd.cut(student_data_df['math_score'], bins, labels=group_names, include_lowest=True)

# Pivot the DataFrame to calculate the average math score for each grade at each school
math_score_by_grade = pd.pivot_table(student_data_df, values='math_score', index='school_name', columns='grade', aggfunc='mean')

# Rename the columns for clarity
math_score_by_grade.columns = [f'Grade {grade} Math Average' for grade in math_score_by_grade.columns]

# Display the DataFrame
print(math_score_by_grade)


                       Grade 10th Math Average  Grade 11th Math Average  \
school_name                                                               
Bailey High School                   76.996772                77.515588   
Cabrera High School                  83.154506                82.765560   
Figueroa High School                 76.539974                76.884344   
Ford High School                     77.672316                76.918058   
Griffin High School                  84.229064                83.842105   
Hernandez High School                77.337408                77.136029   
Holden High School                   83.429825                85.000000   
Huang High School                    75.908735                76.446602   
Johnson High School                  76.691117                77.491653   
Pena High School                     83.372000                84.328125   
Rodriguez High School                76.612500                76.395626   
Shelton High School      

In [47]:

# Define the binning criteria and group names
bins = [0, 59.9, 69.9, 79.9, 89.9, 100]
group_names = ["F", "D", "C", "B", "A"]

# Create a new column in the student_data_df DataFrame to categorize reading scores into bins
student_data_df['Reading Score Bins'] = pd.cut(student_data_df['reading_score'], bins, labels=group_names, include_lowest=True)

# Pivot the DataFrame to calculate the average reading score for each grade at each school
reading_score_by_grade = pd.pivot_table(student_data_df, values='reading_score', index='school_name', columns='grade', aggfunc='mean')

# Rename the columns for clarity
reading_score_by_grade.columns = [f'Grade {grade} Reading Average' for grade in reading_score_by_grade.columns]

# Display the DataFrame
print(reading_score_by_grade)


                       Grade 10th Reading Average  Grade 11th Reading Average  \
school_name                                                                     
Bailey High School                      80.907183                   80.945643   
Cabrera High School                     84.253219                   83.788382   
Figueroa High School                    81.408912                   80.640339   
Ford High School                        81.262712                   80.403642   
Griffin High School                     83.706897                   84.288089   
Hernandez High School                   80.660147                   81.396140   
Holden High School                      83.324561                   83.815534   
Huang High School                       81.512386                   81.417476   
Johnson High School                     80.773431                   80.616027   
Pena High School                        83.612000                   84.335938   
Rodriguez High School       

KeyError: 'Per Student Budget'

In [52]:
# Print the "Per Student Budget" column
print(school_summary.df['Per Student Budget'])


AttributeError: 'DataFrame' object has no attribute 'df'